In [ ]:
# 1. 기준값 추출 (68번 산업)
ref_df = main[
    (main['industry_code'] == 68) &
    main['output_index'].notna() &
    main['labor_productivity_index'].notna()
][['sido', 'base_year_x', 'output_index', 'labor_productivity_index']].copy()

ref_df = ref_df.rename(columns={
    'output_index': 'ref_output_index',
    'labor_productivity_index': 'ref_labor_productivity_index'
})

# 2. 76번 산업 중 결측행 추출
target_idx = main[
    (main['industry_code'] == 76) &
    (main['output_index'].isna() | main['labor_productivity_index'].isna())
].index

target_df = main.loc[target_idx].copy()
target_df['original_index'] = target_df.index  # 인덱스 보존

# 3. merge 후 ref값 가져오기
target_df = target_df.merge(ref_df, on=['sido', 'base_year_x'], how='left')

# 4. 다시 인덱스를 원래 위치로
target_df = target_df.set_index('original_index')

# 5. 원본에 반영 (이제 인덱스 맞음)
main.loc[target_df.index, 'output_index'] = main.loc[target_df.index, 'output_index'].combine_first(target_df['ref_output_index'])
main.loc[target_df.index, 'labor_productivity_index'] = main.loc[target_df.index, 'labor_productivity_index'].combine_first(target_df['ref_labor_productivity_index'])


In [ ]:
# CSV로 저장
main.to_csv('final_사양판단본.csv', index=False, encoding='utf-8-sig')